# Lab 4: Parameter-efficient fine-tuning

Fine-tuning all parameters of pre-trained language models can be resource-intensive. Because of this, current research in natural language processing is looking into developing methods for adapting models to downstream tasks without full fine-tuning. These methods only tune a small number of model parameters while yielding performance comparable to that of a fully fine-tuned model.

In this lab, you will implement LoRA, one of the most well-known methods for parameter-efficient fine-tuning. LoRA stands for “Low-Rank Adaptation of Large Language Models” and was originally described in a research article by [Hu et al. (2021)](https://arxiv.org/abs/2106.09685).

Along the way, you will earn experience with [Hugging Face Transformers](https://huggingface.co/docs/transformers/en/index), a state-of-the-art library for training and deploying language models, as well as with several related libraries. In particular, you will learn a best-practice workflow for downloading a Transformer model and fine-tuning it on the downstream task of binary sentiment classification.

*Tasks you can choose for the oral exam are marked with the graduation cap 🎓 emoji.*

## Dataset

The data for this lab comes from the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). The full dataset consists of 50,000 highly polar movie reviews collected from the Internet Movie Database (IMDB). Here, we use a random sample consisting of 2,000 reviews for training and 500 reviews for evaluation.

To load the dataset, we use the [Hugging Face Datasets](https://huggingface.co/docs/datasets/en/index) library.

In [1]:
from datasets import load_dataset

imdb_dataset = load_dataset(
    "csv", data_files={"train": "train.csv", "eval": "eval.csv"}
)

imdb_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 500
    })
})

As we can see, each sample in the dataset is a record with three fields: an internal index (`index`, an integer), the text of the review (`review`, a string), and the sentiment label (`label`, an integer – 1&nbsp;for “positive” and 0&nbsp;for “negative” sentiment).

Here is an example record:

In [2]:
imdb_dataset["train"][645]

{'index': 2981,
 'review': 'Brilliant execution in displaying once and for all, this time in the venue of politics, of how "good intentions do actually pave the road to hell". Excellent!',
 'label': 1}

## Tokeniser

As our pre-trained language model, we will use [DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert), a compact encoder model with 40% less parameters than BERT base. DistilBERT is not actually a *large* language model by modern standards and thus does not benefit as much from parameter-efficient fine-tuning as other models. However, it has the benefit of being light and fast, and can be run even on consumer hardware.

To feed the movie reviews to DistilBERT, we need to tokenise them and encode the resulting tokens as integers in the model vocabulary. We start by loading the DistilBERT tokeniser using the [Auto classes](https://huggingface.co/docs/transformers/en/model_doc/auto):

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

We then create a tokenised version of the dataset:

In [4]:
def tokenize_function(batch):
    return tokenizer(batch["review"], padding=True, truncation=True)


tokenized_imdb_dataset = imdb_dataset.map(tokenize_function, batched=True)

tokenized_imdb_dataset

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

As we can see, tokenising adds two additional fields to each review: `input_ids` is the list of token ids corresponding to the review, and `attention_mask` is the list of indices specifying which tokens the encoder should attend to.

To avoid trouble when fine-tuning the model later, the next cell disables tokeniser parallelism.

In [5]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Trainer

In this section, we will set up our workflow for training and evaluating DistilBERT models. The central component in this workflow is the [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer), which provides extensive configuration options. Here, we leave most of these options at their default value. Two changes we *do* make are to enable evaluation of the trained model after each epoch, and to log the training and evaluation loss after every 5&nbsp;training steps (the default is 500).

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="tmp_trainer",
    eval_strategy="epoch",
    logging_steps=5,
)

In addition to the loss, we also track classification accuracy. For this we import the [Hugging Face Evaluate](https://huggingface.co/docs/evaluate/en/index) library and define a small helper function `compute_metrics()` that the trainer will call after each epoch.

In [7]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00


In [8]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In the next cell we define a convenience function `make_trainer()` that creates a readily-configured trainer for a specified model (*model*). We will use this trainer both to train the model on the training section of the tokenised review dataset, and to evaluate it on the evaluation section.

In [9]:
from transformers import Trainer


def make_trainer(model):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_imdb_dataset["train"],
        eval_dataset=tokenized_imdb_dataset["eval"],
        compute_metrics=compute_metrics,
    )
    return trainer

## Full fine-tuning

In the rest of this notebook, we will work our way to the implementation of LoRA, and compare LoRA to traditional fine-tuning methods. Our first point of reference is a fully fine-tuned DistilBERT model.

We start by loading the pre-trained model:

In [10]:
from transformers import AutoModelForSequenceClassification

pretrained_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)

pretrained_model

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


The architecture of DistilBERT is that of a standard Transformer encoder with an embedding layer (`embeddings`) followed by a stack of six Transformer blocks (`transformer`) and a feedforward network with two linear layers (`pre_classifier` and `classifier`) and a final dropout layer (`dropout`).

### 🎈 Task 4.01: Counting the number of trainable parameters

One relevant measure in the context of parameter-efficient fine-tuning is the number of parameters that need to be changed when training a model. Your first task in this lab is to write a function `num_trainable_parameters()` that calculates this number for a given model.

In [11]:
def num_trainable_parameters(model):
    the_sum = 0
    for param in model.parameters():
      if param.requires_grad:
        the_sum += param.numel()
    return the_sum

In [12]:
num_trainable_parameters(pretrained_model)

66955010

The function should implement the following specification:

> **num_trainable_parameters** (*model*)
>
> Returns the number of float-valued trainable parameters in the specified *model* as an integer.

#### 👍 Hint

The term *parameter* can refer to either complete tensors or the individual elements of these tensors. For example, a linear layer created by `nn.Linear(3, 5)` has 2&nbsp;tensor-valued parameters (a weight matrix and a bias vector) and 20&nbsp;float-valued parameters (the elements of these tensors). To get the tensor-valued parameters of a model, you can use the [`parameters()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters) method. A parameter is *trainable* if it requires gradient.

#### 🤞 Test your code

To test your code, apply your function to the pre-trained model. The correct number of float-valued trainable parameters for this model is 66,955,010.

### Fine-tuning

When we load the pre-trained model, the Hugging Face Transformers library warns us that the weights of the feedforward network have not yet been trained. To do so, in the next cell, we pass the pre-trained model to a trainer and initiate the fine-tuning process.

**⚠️ Please note that fine-tuning the model will take some time! ⚠️**

You can work on the other problems in this lab while you are waiting.

In [13]:
finetuned_trainer = make_trainer(pretrained_model)

finetuned_trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahmadabdou963 (ahmadabdou963-linn-universitetet) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.496500,0.334950,0.894000
2,0.059300,0.305215,0.910000
3,0.003400,0.348030,0.918000


TrainOutput(global_step=750, training_loss=0.22576220335811376, metrics={'train_runtime': 338.0783, 'train_samples_per_second': 17.747, 'train_steps_per_second': 2.218, 'total_flos': 794804391936000.0, 'train_loss': 0.22576220335811376, 'epoch': 3.0})

Because full fine-tuning is so resource-intensive, we save the fine-tuned model to disk:

In [14]:
finetuned_trainer.save_model("finetuned")

Later in this notebook, whenever you need the fully fine-tuned version of the model, you can load it as follows:

In [15]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained("finetuned")

### Convenience functions

Because we will repeat the steps we just took to fine-tune the pre-trained model several times in this notebook, we define two convenience functions:

In [16]:
def train(model):
    print("Number of trainable parameters:", num_trainable_parameters(model))
    trainer = make_trainer(model)
    trainer.train()
    return model

In [17]:
def evaluate(model):
    trainer = make_trainer(model)
    return trainer.evaluate()

## Tuning the final layers only

If full fine-tuning marks one end of the complexity spectrum, the other end is marked by only tuning the final layers of the transformer – the *head* of the model. In the case of DistilBERT, the head consists of the `pre_classifier` and `classifier` layers.

### 🎈 Task 4.02: Head-tuning

Implement the head-tuning strategy by coding the following function:

In [20]:
from transformers import AutoModelForSequenceClassification
def make_headtuned_model():
  model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

  for param in model.paramters():
    param.requires.grad = False

  for param in model.pre_classifier.parameters():
    param.requires_grad = True

  for param in model.classifier.paramteres():
    param.requires_grad = True

    raise model

Here is the specification of this function:

> **make_headtuned_model** ()
>
> Returns a model that is identical to the pre-trained model, except that the head layers have been trained on the sentiment data. (The other parameters of the pre-trained model are left untouched.)

#### 👍 Hint

You freeze a parameter by setting its `requires_grad`-attribute to `False`.

Once you have an implementation of the head-tuning strategy, evaluate it on the evaluation data. How much accuracy do we lose when only training the final layers of the pre-trained model, compared to full fine-tuning?

In [19]:
headtuned_model = make_headtuned_model()

NotImplementedError: 

#### 🤞 Test your code

If you configured your model correctly, `num_trainable_parameters()` should show 592,130 trainable parameters.

For future reference, we also save the head-tuned model:

In [ ]:
make_trainer(headtuned_model).save_model("headtuned")

## Layer surgery

LoRA works by “wrapping” frozen layers from the pre-trained Transformer model inside adapter modules. Conventionally, this wrapping is only applied to the linear layers that transform the queries and values in the self-attention mechanism. To implement the wrapping, we need functions to extract and replace layers in a model. Your task in this section is to code these functions.

### 🎓 Task 4.03: Extracting layers

Code a function that extracts the query and value linear layers from a DistilBERT model:

In [ ]:
def extract(model):
    # TODO: Replace the next line with your own code
    return {}

Implement this function to match the following specification:

> **extract** (*model*)
>
> Takes a DistilBERT model (*model*) and extracts the query and value linear layers from each block of the Transformer. Returns a dictionary mapping the DistilBERT module names of these layers to the layers themselves (instances of `nn.Linear`).

#### 👍 Hint

As we saw earlier, the DistilBERT model consists of a hierarchy of nested submodules. Each of these can be addressed by a fully-qualified string name. Use [`get_submodule()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.get_submodule) to retrieve a layer by name. You can hard-wire the names of the layers you want to extract.

#### 🤞 Test your code

To test your code, check the number of trainable float-valued parameters in the extracted layers. This number should be 7,087,104.

### 🎓 Task 4.04: Replacing layers

Next, code the inverse of the `extract()` function to replace selected layers of a module using a dictionary of named layers.

In [ ]:
def replace(model, named_layers):
    # TODO: Replace the next line with your own code
    return model

Implement this function to match the following specification:

> **replace** (*model*, *named_layers*)
>
> Takes a DistilBERT model (*model*) and a dictionary in the format returned by `extract()` (*named_layers*) and injects the extracted layers into the model. More specifically, suppose that *named_layers* contains a key–value pair `(name, layer)`. Then the function replaces the submodule of *model* addressed by the fully-qualified string name `name` by the layer `layer`. Returns the modified model.

#### 👍 Hint

Use [`getattr()`](https://docs.python.org/3/library/functions.html#getattr) and [`setattr()`](https://docs.python.org/3/library/functions.html#setattr) to return or set the value of a named submodule.

#### 🤞 Test your code

To test your implementation, write code that (1)&nbsp;extracts the query and value linear layers from the fine-tuned model; (2)&nbsp;replaces these layers with clones with random weights; and (3)&nbsp;replaces these layers again with the original versions. Evaluating the modified model after step&nbsp;(2) should yield a near-random accuracy. Evaluating it again after step&nbsp;(3) should yield the original accuracy.

The following function should be helpful. It clones a linear layer, copying the weights and the bias from the original.

In [ ]:
import torch.nn as nn


def clone_linear(original):
    out_features, in_features = original.weight.shape
    copy = nn.Linear(in_features, out_features)
    copy.load_state_dict(original.state_dict())
    return copy

## Low-rank approximation

The basic idea behind LoRA is to conceptualise fine-tuned weights as a sum $W_0 + \Delta W$ of the weights from the pre-trained model, $W_0$, and a low-rank update matrix $\Delta W$. The goal of fine-tuning, then, is to learn the update matrix; this happens in the adapter layers.

Before we get to the implementation of the LoRA adapter layers, we first check to what extent the assumption that fine-tuning can be described by low-rank matrices holds true for DistilBERT. To do so, we will “cheat” and replace the query and value linear layers of the head-tuned model with low-rank approximations. The technical key to this is the truncated singular value decomposition (SVD).

### 🎓 Task 4.05: Low-rank matrix approximation

Your first task in this section is to implement the low-rank matrix approximation.

In [ ]:
def approximate(matrix, rank):
    # TODO: Replace the next line with your own code
    return matrix

Implement this function to match the following specification:

> **approximate** (*matrix*, *rank*)
>
> Takes a 2D-tensor (*matrix*) and an integer rank $r$ (*rank*), computes the truncated SVD with rank $r$ on the tensor, and returns the corresponding low-rank approximation matrix.

#### 👍 Hint

If you need a refresher on the low-rank matrix approximation, read the corresponding section from the Wikipedia article on the [Singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition#Low-rank_matrix_approximation). The truncated SVD is an extension of the full SVD; the latter can be computed using [`torch.linalg.svd()`](https://pytorch.org/docs/stable/generated/torch.linalg.svd.html).

#### 🤞 Test your code

To test your code, run the following cell. It creates a matrix `original` with rank $r \leq 8$ and after that the rank-$8$ approximation matrix `approximation`. You should find that the distance between the two matrices is very low.

In [ ]:
original = torch.rand(768, 8) @ torch.rand(8, 384)
approximation = approximate(original, 8)
torch.dist(original, approximation)

### 🎓 Task 4.06: Approximated fine-tuned model (version 1)

In the next step, your task is to construct a version of the head-tuned model in which every query and value linear layer is replaced by a low-rank approximation of the corresponding layer from the fully fine-tuned model.

In [ ]:
def make_approximated_model_1(rank):
    # TODO: Replace the next line with your own code
    raise NotImplementedError

Here is the specification of this function:

> **make_approximated_model_1** (*rank*)
>
> Takes an integer rank $r$ (*rank*) and returns a version of the head-tuned model in which every query and value linear layer is replaced by its $r$-approximated corresponding layer from the fully fine-tuned model.

Run the next cell to evaluate your model for different rank values. Start with the full rank and then halve the rank in each step. What is the lowest rank that still gives you a higher accuracy than the head-tuned model?

In [ ]:
approximated_model_1 = make_approximated_model_1(768)

evaluate(approximated_model_1)

### 🎓 Task 4.07: Approximated fine-tuned model (version 2)

In the approximated model from the previous section, the truncated SVD is applied to the full weight matrix of the fine-tuned model: $W_0 + \Delta W$. In LoRA, the low-rank approximation only applies to the *update matrix* $\Delta W$, i.e., the difference between the fully fine-tuned weights and the pre-trained weights.

In [ ]:
def make_approximated_model_2(rank):
    # TODO: Replace the next line with your own code
    raise NotImplementedError

Implement the function to match the following specification:

> **make_approximated_model_2** (*rank*)
>
> Takes an integer rank $r$ (*rank*) and returns a version of the head-tuned model in which the weight matrix of every query and value linear layer is replaced by the sum $W_0 + \Delta W$, where $W_0$ is the weight matrix of the pre-trained model and $\Delta W$ is the rank-$r$ approximation of the update matrix, i.e., the difference between the fully fine-tuned weights and the pre-trained weights.

Run the next cell to evaluate your model for different rank values. Start with the rank from the approximated model from the previous section and then halve the rank in each step. What is the lowest rank that still gives you a higher accuracy than the head-tuned model?

In [ ]:
approximated_model_2 = make_approximated_model_2(768)

evaluate(approximated_model_2)

In [ ]:
approximated_model_2 = make_approximated_model_2(3)

evaluate(approximated_model_2)

## Low-Rank Adaptation (LoRA)

In this section, you will implement the LoRA adapters and fine-tune the adapted model.

### 🎓 Task 4.08: Implement the adapter

A LoRA adapter implements the forward function

$$
y = x W_0 + x \Delta W = x W_0 + x A B
$$

where $W_0$ is a linear transformation from the pre-trained model and $\Delta W$ is a learned update matrix, deconstructed into the product $AB$ of two rank-$r$ matrices $A$ and $B$. LoRA scales the update matrix $\Delta W$ by a factor of $\alpha / r$, where $\alpha$ is a hyperparameter. (To keep the formula tidy, we ignore the fact that the linear transformation in the pre-trained model may additionally include a bias.)

In [ ]:
import torch.nn as nn


class LoRA(nn.Module):
    def __init__(self, pretrained, rank=12, alpha=24):
        super().__init__()
        # TODO: Add your code here

    def forward(self, x):
        # TODO: Replace the next line with your own code
        raise NotImplementedError

Your code must comply with the following specification:

**__init__** (*self*, *pretrained*, *rank* = 12, *alpha* = 24)

> Initialises the LoRA adapter. This sets up the matrices $A$ and $B$ from the equation above. The matrix $A$ is initialised with random weights from a standard normal distribution; the matrix $B$ is initialised with zeros. The argument *pretrained* is the linear layer from the pre-trained model that should be adapted. The arguments *rank* and *alpha* are the rank $r$ and the hyperparameter $\alpha$ in the equation above.

**forward** (*self*, *x*)

> Sends an input *x* through the adapter, implementing the equation above.

### 🎓 Task 4.09: Inject the adapter into the pre-trained model

The final step is to construct an adapted model by injecting the LoRA adapters into the pre-trained model.

In [ ]:
def make_lora_model(rank):
    # TODO: Replace the next line with your own code
    raise NotImplementedError

Implement the function to match the following specification:

> **make_lora_model** (*rank*)
>
> Returns a model that is identical to the pre-trained model, except that the query and value linear layers have been wrapped in LoRA adapters, and the LoRA adapters and the head layers of the pre-trained model have been trained on the sentiment data. (The other parameters of the pre-trained model are left untouched.) The rank of the adapters is specified by the argument *rank*. The *alpha* value of the adapters is set to twice the rank (a common rule of thumb).

Run the next cell to evaluate your model for $r = 6$ and $\alpha = 12$. How many trainable parameters does the adapted model have? What accuracy do you get? How do these value relate to the number of trainable parameters and accuracy of the fully fine-tuned model, in terms of percentages?

In [ ]:
lora_model = make_lora_model(6)

evaluate(lora_model)

## Alternatives to Transformer-based models

Even with methods for parameter-efficient fine-tuning, applying DistilBERT and other Transformer-based models comes at a significant cost – an investment that does not always pay off. In the final task of this lab, we ask you to explore a more traditional approach to classification and contrast it with the pre-training/fine-tuning approach of neural language models.

### 🎓 Task 4.10: Comparing with a non-neural classifier

Browse the web to find a tutorial on how to apply a classifier from the [scikit-learn](https://scikit-learn.org/stable/) library to the problem of sentiment classification and implement the method here in this notebook. We suggest you use [multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) or [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). (Once you have code for one method, it is easy to switch to the other.) Evaluate your chosen classifier on the IMDB dataset.

Questions to consider:

* Which classifier did you try? What results did you get? How long did it take you to train and run the classifier?
* What is your perspective on the trade-off between accuracy and resource requirements between the two approaches?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

**🥳 Congratulations on finishing this lab! 🥳**